# MCMC0.5: Introduction to Functional Programming

## Why functional?

One may claim that C/Fortran (or sometimes C++ without functional programming extension) is enough to do physics because they are fast, simple, and have enough functionalities. I agree with all the three points, but I think it is acceptable only when you are working only by yourself. Usually, a code by procedural programming is unreadable and not human-oriented. In the procedural programming, you always have to use your brain to mimic how the computer works (is a human being a slave of computers?). Most computational physicists somehow got used to this way of thinking (already brainwashed) and wrote a code as readable to the machine as possible. Of course, it is ok if you are working by yourself and your collaborators do not care about your code, but today it is more important to share a code (via Github) and discuss physics on the Jupyter Notebook (Maybe in old days it was enough to have one expert for the computation and the others did not even have to see the code). In that case, it is important to write a readable, understandable, and well-organized code for everybody even without any comments!

I want to define functional programming as human-oriented programming. If you want to get a mean value, why not apply a "mean" function onto the collection? Some (brainwashed) people may wonder if it is ok not to sum each time of the iteration to reduce memory, and so on. Such concerns will be resolved later by a concept of lazy evaluation (actually in MCMC6.0). Using this black box, you now do not have to care about at which step functions are actually called. From now on, you can just apply statistical operations like mean, std, etc. just on the collection of data (called iterator) afterwards. This is exactly how we think when we process big data, and you can now write a code as you think.

More strictly, functional programming may be defined as programming writing a code by compositions of pure functions. In other words, it is a deterministic finite-state machine (FSM). This notion is closely related to the Markov-chain Monte Carlo (MCMC) because MCMC is nothing but an underteministic FSM. From physicists' view, it is a "quantum" version of the functional programming, and every techinique in functional programming is applicable to MCMC, while in MCMC completely pure functional programming is impossible because we need some randomness (isn't it exactly like quantum mechanics?). From now on, I will take "pragmatism" by using functional programming as much as possible, but by including a nondeterministic function additionally to make it work as MCMC. In this sense, I do not stick to doing completely functional programming, but use it as a tool to make a code readable. Unless it diminishes the readability of a code, I sometimes use a destructive function only if needed.

Julia completely enables us to write a code as you think. You can use Unicode for variables, most linear algebra functions already exist, and Julia even discards class-based object-oriented programming, which is unnecessary for MCMC! Let's see the HMC function in MCMC1.0, and compare the function to the algorithm written above the function. If you are good at elementary mathematics, you can write a code exactly like you are doing math on your notebook. Like in [this pdf](https://github.com/MCSMC/MCMC_sample_codes/blob/master/review-MCMC.pdf), people unfortunately still believe that such "how we think" programming in Mathematica is bad. However, today in 2018, Julia and Jupyter Notebook work as a super-fast version of Mathematica and Julia now works at speed comparable to C/Fortran. Of course, if you wish to use low-level operations like SSE and AVX, we still need such procedural languages, but for most purposes, just calling BLAS/LAPACK functions is enough. Even more, Julia now supports GPGPU, so even for GPGPU programming Julia can be the first choice, instead of writing CUDA/OpenCL directly.

In this series of notebooks, I will introduce not only MCMC but also functional programming, Julia-like programming, and in addition a Bayesian way of thinking. Think Bayesian, think functional, and last but not least think Julia!

## map

The most important higher-order function is map. See how it works.

In Julia you won't see this function not many times because there is a very nice short-hand notation.

The problem is that map in Julia does not do lazy evaluation. Each time we call map arrays are stored in the memory.

In order to do lazy evaluation, use Base.Generator instead.

I will come back to this point later in MCMC2.0 and MCMC6.0.

## filter

The next important higher-order function is filter.

## reduce

## foreach

map, filter and reduce were the three most important functions. Compared to those, foreach is not important because it can be written instead by a "for loop." foreach just reduces the length of your code.

## Currying